In [84]:
""" Ingest additional rasters like DEM, LDD etc. 
-------------------------------------------------------------------------------
This notebook will upload the geotiff files of auxiliary rasters 
from S3 to Google Cloud Storage and into the WRI/aqueduct earthengine bucket. 

Requirements:
    Authorize earthengine by running in your terminal: earthengine 
                                                       authenticate

    you need to have access to the WRI-Aquaduct (yep a Google employee made a
    typo) bucket to ingest the data. Rutger can grant access to write to this 
    folder. 

    Have access to the Google Cloud Storage Bucket
    
    AWS CLI configured

Make sure to set the project to Aqueduct30 by running
`gcloud config set project aqueduct30`

Code follows the Google for Python Styleguide. Exception are the scripts that 
use earth engine since this is camelCase instead of underscore.

Author: Rutger Hofste
Date: 20170802
Kernel: python27
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:    
    TESTING (Boolean) : Toggle Testing Mode.
    OVERWRITE (Boolean) : Overwrite old folder !CAUTION!
    SCRIPT_NAME (string) : Script name.  
    S3_INPUT_PATH (string) : Amazon S3 location of data to be ingested.
    X_DIMENSION_5MIN (integer) : horizontal or longitudinal dimension of 
                                 raster.
    Y_DIMENSION_5MIN (integer) : vertical or latitudinal dimension of 
                                 raster.
    GCS_BASE (string) : Google Cloud Storage output namespace.   
    EE_BASE (string) : Earth Engine folder to store the assets.
    
    OUTPUT_FILE_NAME (string) : File Name for a csv file containing the failed tasks. 
    
Returns:


"""

# Input Parameters
TESTING = 0
OVERWRITE = 0 # !CAUTION!
SCRIPT_NAME = "Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02"
S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/rawData/Utrecht/additionalFiles/flowNetwork/topo_pcrglobwb_05min"
X_DIMENSION_5MIN = 4320
Y_DIMENSION_5MIN = 2160
GCS_BASE = "gs://aqueduct30_v01/{}/".format(SCRIPT_NAME)
EE_BASE = "projects/WRI-Aquaduct/PCRGlobWB20V08"
OUTPUT_FILE_NAME = "df_errorsV01.csv"


In [33]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

('Y2018M04D13', 'UTC 10:28')


'2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 17:05:42) \n[GCC 7.2.0]'

In [71]:
# imports
import os
import numpy as np
import subprocess
import warnings
import pandas as pd
from osgeo import gdal
from datetime import timedelta

import aqueduct3

In [53]:
# ETL

ec2_input_path = "/volumes/data/{}/input".format(SCRIPT_NAME)
ec2_output_path = "/volumes/data/{}/output".format(SCRIPT_NAME)

s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output/".format(SCRIPT_NAME)

In [37]:
!mkdir -p {ec2_input_path}
!mkdir -p {ec2_output_path}

In [36]:
!aws s3 cp {S3_INPUT_PATH} {ec2_input_path} --recursive

download: s3://wri-projects/Aqueduct30/rawData/Utrecht/additionalFiles/flowNetwork/topo_pcrglobwb_05min/geotiff/cellsize05min.tif to ../../../../data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/input/geotiff/cellsize05min.tif
download: s3://wri-projects/Aqueduct30/rawData/Utrecht/additionalFiles/flowNetwork/topo_pcrglobwb_05min/geotiff/readme.txt to ../../../../data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/input/geotiff/readme.txt
download: s3://wri-projects/Aqueduct30/rawData/Utrecht/additionalFiles/flowNetwork/topo_pcrglobwb_05min/geotiff/lddsound_05min.tif to ../../../../data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/input/geotiff/lddsound_05min.tif
download: s3://wri-projects/Aqueduct30/rawData/Utrecht/additionalFiles/flowNetwork/topo_pcrglobwb_05min/geotiff/accumulated_drainage_area_05min_sqkm.tif to ../../../../data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/input/geotiff/accumulated_drainage_area_05min_sqkm.tif
download: s3://wri-projects/Aqueduct30/rawData

renaming the files with a structured name using schema: `<indicator>_<spatial_resolution>_<unit>.map`

copy from readme.txt file on S3


Data received from Rens van Beek on Feb 24 2017. Rutger Hofste
topo_pcrglobwb_05min.zip  

|Archive Length |   Date |   Time |    Name | Units | newName | 
|---:|---:|---:|---:| ---:| ---:|
|37325056|  02-24-2017| 15:46 |  accumulated_drainage_area_05min_sqkm.map| $$km^2$$ |accumulateddrainagearea_05min_km2.map |
|37325056|  02-24-2017 |15:45 |  cellsize05min.correct.map| $$m^2$$ |cellsize_05min_m2.map |
|37325056| 02-24-2017| 15:44 |  gtopo05min.map| $$m$$ |gtopo_05min_km2.map|
|9331456|  02-24-2017| 15:45 |   lddsound_05min.map| numpad |lddsound_05min_numpad.map |
|121306624| | |               4 files| | |

All files are 5 arc minute maps in PCRaster format and WGS84 projection (implicit).
The gtopo05min.map is the DEM from the gtopo30 dataset that we use for downscaling meteo data occasionally. This is consistent with the CRU climate data sets and the hydro1k drainage dataset. Elevation is in **metres**. The cellsize05.correct.map is the surface area of a geographic cell in **m2** per 5 arc minute cell. The lddsound_05min.map is identical to the LDD we sent you earlier with the **8-point pour algorithm**.(numpad e.g. 7 is NW 6 is E etc.) The accumulated_drainage_area_05min_sqkm.map is the accumulated drainage area in **km2** per cell along the LDD.


In [38]:
!mv {ec2_input_path}/accumulated_drainage_area_05min_sqkm.map \
{ec2_input_path}/accumulateddrainagearea_05min_km2.map
!mv {ec2_input_path}/cellsize05min.correct.map \
{ec2_input_path}/cellsize_05min_m2.map
!mv {ec2_input_path}/gtopo05min.map \
{ec2_input_path}/gtopo_05min_m.map
!mv {ec2_input_path}/lddsound_05min.map \
{ec2_input_path}/lddsound_05min_numpad.map

In [39]:
# Added on 20180413. Removing geotiff folder since the extent was generated by arcgis and is not
# exactly 180 and 90 degrees respectively
!rm -r {ec2_input_path}/geotiff

In [63]:
# Functions
def get_GCS_keys(GCS_BASE):
    """ get list of keys from Google Cloud Storage
    -------------------------------------------------------------------------------
    
    Args:
        GCS_BASE (string) : Google Cloud Storage namespace containing files.
        
    Returns:
        df (pd.DataFrame) : DataFrame with properties useful to Aqueduct. 
    
    """
    command = "/opt/google-cloud-sdk/bin/gsutil ls {}".format(GCS_BASE)
    keys = subprocess.check_output(command,shell=True)
    keys = keys.decode('UTF-8').splitlines()
    
    df = keys_to_df(keys)
    
    return df

def keys_to_df(keys):
    """ helper function for 'get_GCS_keys'
    
        
    Args:
        keys (list) : list of strings with keys.
        
    Returns:
        df (pd.DataFrame) : Pandas DataFrame with all relvant properties for
                            Aqueduct 3.0.
    """
    
    df = pd.DataFrame()
    i = 0
    for key in keys:
        i = i+1
        schema = ["indicator","spatial_resolution","unit"]
        out_dict = aqueduct3.split_key(key,schema)
        df2 = pd.DataFrame(out_dict,index=[i])
        df = df.append(df2)    
    return df


In [40]:
# Convert rasters to geotiff

In [41]:
default_geotransform, default_geoprojection = aqueduct3.get_global_georeference(np.ones([Y_DIMENSION_5MIN,X_DIMENSION_5MIN]))

In [49]:
for root, dirs, file_names in os.walk(ec2_input_path):
    for file_name in file_names:
        print(file_name)
        base , extension = file_name.split(".")
        if extension == "map":
            output_path = os.path.join(ec2_output_path,base + ".tif")
            input_path = os.path.join(root, file_name)     

            xsize,ysize,geotransform,geoproj,Z = aqueduct3.read_gdal_file(input_path)
            Z[Z<-9990]= -9999
            Z[Z>1e19] = -9999
            aqueduct3.write_geotiff(output_path,default_geotransform,default_geoprojection,Z,nodata_value=-9999,datatype=gdal.GDT_Float32)
        else:
            warnings.warn("File Not Converted: {}".format(file_name), UserWarning)

gtopo_05min_m.map
readme.txt
lddsound_05min_numpad.map


/opt/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:14: UserWarning: File Not Converted: readme.txt
  


accumulateddrainagearea_05min_km2.map
cellsize_05min_m2.map


In [51]:
!gsutil -m cp \
{ec2_output_path}/*.tif \
{GCS_BASE}

Copying file:///volumes/data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/cellsize_05min_m2.tif [Content-Type=image/tiff]...
Copying file:///volumes/data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/gtopo_05min_m.tif [Content-Type=image/tiff]...
Copying file:///volumes/data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/lddsound_05min_numpad.tif [Content-Type=image/tiff]...
Copying file:///volumes/data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/accumulateddrainagearea_05min_km2.tif [Content-Type=image/tiff]...
| [4/4 files][ 16.0 MiB/ 16.0 MiB] 100% Done                                    
Operation completed over 4 objects/16.0 MiB.                                     


In [54]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive

upload: ../../../../data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/cellsize_05min_m2.tif to s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/cellsize_05min_m2.tif
upload: ../../../../data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/lddsound_05min_numpad.tif to s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/lddsound_05min_numpad.tif
upload: ../../../../data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/accumulateddrainagearea_05min_km2.tif to s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/accumulateddrainagearea_05min_km2.tif
upload: ../../../../data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/gtopo_05min_m.tif to s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/output/gtopo_05min_m.tif


In [64]:
df = get_GCS_keys(GCS_BASE)
df.shape

(4, 5)

In [67]:
df_complete = df.copy()

In [68]:
df_complete["nodata_value"] = -9999
df_complete["ingested_by"] ="RutgerHofste"
df_complete["exportdescription"] = df_complete["indicator"]
df_complete["script_used"] = SCRIPT_NAME
df_complete = df_complete.apply(pd.to_numeric, errors='ignore')

In [88]:
df_errors = pd.DataFrame()
start_time = time.time()
for index, row in df_complete.iterrows():
    elapsed_time = time.time() - start_time 
    print(index,"{:02.2f}".format((float(index)/df_complete.shape[0])*100) + "elapsed: ", str(timedelta(seconds=elapsed_time)))
    
    geotiff_gcs_path = GCS_BASE + row.file_name + "." + row.extension
    output_ee_asset_id = EE_BASE +"/"+ row.file_name
    properties = row.to_dict()
    
    df_errors2 = aqueduct3.upload_geotiff_to_EE_imageCollection(geotiff_gcs_path, output_ee_asset_id, properties,index)
    df_errors = df_errors.append(df_errors2)

(1, '25.00elapsed: ', '0:00:00.000614')
/opt/anaconda3/bin/earthengine upload image --asset_id projects/WRI-Aquaduct/PCRGlobWB20V08/accumulateddrainagearea_05min_km2 gs://aqueduct30_v01/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/accumulateddrainagearea_05min_km2.tif -p indicator=accumulateddrainagearea -p extension=tif -p file_name=accumulateddrainagearea_05min_km2 -p ingested_by=RutgerHofste -p script_used=Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02 -p exportdescription=accumulateddrainagearea -p spatial_resolution=05min -p unit=km2 --nodata_value=-9999
(2, '50.00elapsed: ', '0:00:03.262118')
/opt/anaconda3/bin/earthengine upload image --asset_id projects/WRI-Aquaduct/PCRGlobWB20V08/cellsize_05min_m2 gs://aqueduct30_v01/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02/cellsize_05min_m2.tif -p indicator=cellsize -p extension=tif -p file_name=cellsize_05min_m2 -p ingested_by=RutgerHofste -p script_used=Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V02 -p exportdescription=cellsize

In [89]:
df_errors.to_csv("{}/{}".format(ec2_output_path,OUTPUT_FILE_NAME))

In [91]:
df_errors

,command,error,response
1,/opt/anaconda3/bin/earthengine upload image --...,0,Started upload task with ID: 76UC5LNKN6PAB44ZI...
2,/opt/anaconda3/bin/earthengine upload image --...,0,Started upload task with ID: K6M4PZQCLVZOTW3MR...
3,/opt/anaconda3/bin/earthengine upload image --...,0,Started upload task with ID: KIBMER3MLXH4BOQAQ...
4,/opt/anaconda3/bin/earthengine upload image --...,0,Started upload task with ID: PVXEAG5TVML2WYNVL...


In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)